In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

In [ ]:
data = pd.read_excel("test.xlsx")
drop_num = []  # data preprocessing, remove odd data

for i, val in enumerate(data["I"]):
    if (i > 0) and (data["V_T"][i - 1] > data["V_T"][i]) == 1:
        drop_num.append(i)

    if (val < 0) or (val > 9e-6):
        drop_num.append(i)
data.drop(drop_num, inplace=True)

In [ ]:
groups = data.groupby("cell No.")
pieces = [group for _, group in groups]
splitted_pieces = np.array_split(pieces, 6)

In [ ]:
def model_function(x, params):
    # Example function: a * x^2 + b * x + c
    Is, n = params
    return Is * (np.exp(x / n * 0.026) - 1)


def residual_function(params, x, y):
    return model_function(x, params) - y


result_Is = []
result_n = []
for i, piece in enumerate(splitted_pieces):
    x = splitted_pieces[i][0]["V_T"]
    y = splitted_pieces[i][0]["I"]
    initial_params = [1e-9, 1]
    result = least_squares(residual_function, initial_params, args=(x, y), method="lm")
    result_Is.append(result.x[0])
    result_n.append(result.x[1])

In [ ]:
print(np.average(result_Is))
print(np.average(result_n))